In [1]:
%matplotlib inline
projecttitle = 'Analogy'
import sys, os
if sys.platform == 'darwin':
    sys.path.append(os.path.join("/Users", "njchiang", "GitHub", "task-fmri-utils"))
else:
    sys.path.append(os.path.join("D:\\", "GitHub", "task-fmri-utils"))

In [4]:
from fmri_core import analysis as pa
from fmri_core import utils as pu

In [7]:
os.chdir("D:\CloudStation\Grad\Research\montilab-ucla")

In [8]:
projectSettings = pu.loadConfig(os.path.join('analogy', 'config', 'project.json'))
analysisSettings = pu.loadConfig(os.path.join('analogy', 'config', 'analyses.json'))

Loading JSON config from analogy\config\project.json
Loading JSON config from analogy\config\analyses.json


In [25]:
projectSettings['subjects'].keys()

dict_keys(['sub-01', 'sub-02', 'sub-03', 'sub-04', 'sub-05', 'sub-06', 'sub-07', 'sub-08', 'sub-09', 'sub-10', 'sub-11', 'sub-12', 'sub-13', 'sub-14', 'sub-15', 'sub-16'])

In [9]:
paths = projectSettings['filepaths']['winPaths']

In [18]:
# for each subject
# mask = aal-LOFC-bin_mask
def runsubject(sub, mask):
    # set image data
    imgFile = os.path.join(paths['root'], 'derivatives', sub, 'betas', pu.formatBIDSName(sub, 'task-analogy', 'betas-pymvpa.nii.gz'))
    mask = pu.loadImg(paths['root'], 
                      'derivatives', sub, 'masks', 
                      mask + '.nii.gz')
    labels = pu.loadLabels(paths['root'], 
                           'derivatives', sub, 'betas', 
                           pu.formatBIDSName(sub, 'task-analogy', 'events-pymvpa.tsv'), 
                           sep='\t', index_col=0)
    # load image
    maskedImg = pu.maskImg(imgFile, mask)
    # clean out timepoints of interest
    conditionSelector = labels['ab'] == 1

    # preprocessing
    from sklearn.preprocessing import StandardScaler
    op = StandardScaler()
    scaledData = pa.opByLabel(maskedImg, labels['chunks'], op)
    # analysis
    from sklearn.svm import SVC
    clf = SVC()
    from sklearn.model_selection import LeaveOneGroupOut
    
    cv = LeaveOneGroupOut().split(scaledData[conditionSelector], labels['trialtype'][conditionSelector], labels['chunks'][conditionSelector])
    result = pa.roi(scaledData[conditionSelector], labels['abmainrel'][conditionSelector], clf, cv=cv) #, groups=labels['chunks'][conditionSelector])
    # result = pa.unmaskImg(scaledData, mask)
    return result


In [36]:
test = runsubject('sub-01', 'svm-abmainrel-bin_mask')
test.mean()

Reading file from: D:\fmri\Analogy\derivatives\sub-01\masks\svm-abmainrel-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\betas\sub-01_task-analogy_events-pymvpa.tsv
Masking D:\fmri\Analogy\derivatives\sub-01\betas\sub-01_task-analogy_betas-pymvpa.nii.gz
applying operation by label


0.37847222222222221

In [37]:
maskRes = [runsubject(s, 'svm-abmainrel-bin_mask') 
           for s in projectSettings['subjects'].keys()]

Reading file from: D:\fmri\Analogy\derivatives\sub-01\masks\svm-abmainrel-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-01\betas\sub-01_task-analogy_events-pymvpa.tsv
Masking D:\fmri\Analogy\derivatives\sub-01\betas\sub-01_task-analogy_betas-pymvpa.nii.gz
applying operation by label
Reading file from: D:\fmri\Analogy\derivatives\sub-02\masks\svm-abmainrel-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-02\betas\sub-02_task-analogy_events-pymvpa.tsv
Masking D:\fmri\Analogy\derivatives\sub-02\betas\sub-02_task-analogy_betas-pymvpa.nii.gz
applying operation by label
Reading file from: D:\fmri\Analogy\derivatives\sub-03\masks\svm-abmainrel-bin_mask.nii.gz
Loading label file from: D:\fmri\Analogy\derivatives\sub-03\betas\sub-03_task-analogy_events-pymvpa.tsv
Masking D:\fmri\Analogy\derivatives\sub-03\betas\sub-03_task-analogy_betas-pymvpa.nii.gz
applying operation by label
Reading file from: D:\fmri\Analogy\derivatives\sub-04\masks\svm-abm

In [38]:
import numpy as np
np.array(maskRes).mean(), np.array(maskRes).std()/np.sqrt(16)

(0.37022569444444442, 0.017707256209582221)

In [49]:
np.array(maskRes).mean() - 2.5*np.array(maskRes).mean(1).std()/np.sqrt(16)

0.35226664782114209

In [47]:
np.array(maskRes).mean(1).std()/np.sqrt(16)*2

0.014367237298641871

(16,)